## Pandas

In [ ]:
import pandas as pd

### pd.Series

`pd.Series` — это аналог массива `np.ndarray`, но в качестве индекса можно использовать любые объекты.
Так же как и `np.ndarray` поддерживает векторизированные операции

In [ ]:
s1 = pd.Series(range(10))
s2 = pd.Series(range(10, 20))
s1 + s2

#### Нечисловой индекс

In [ ]:
s3 = pd.Series({'x': 132, 'y': 456})
s4 = pd.Series({'z': 100, 'x': -100})

s3 * s4

#### Типы данных
- `str`
- `float` различных размеров
- `int` различных размеров
- `bool`
- `categorical`

In [ ]:
marks = pd.CategoricalDtype(['BMW', 'Volvo', 'Mercedes'])
autos = pd.Series(['BMW', 'BMW', 'Volvo', 'Mercedes'] * 10)
converted = autos.astype(marks)

print(autos.memory_usage())
print(converted.memory_usage())

#### Операции для работы со строками
Доступны через `pd.Series.str`, эффективно реализованы, в том числе регулярные вырожеения

In [ ]:
sentences = pd.Series([
    'Маша мыла раму',
    'Проезд стоит 33 рубля',
    'Просто какой-то достаточно длинный текст на руксском языке',
])

print('Длины текстов')
print(sentences.str.len())

print('Тексты в нижнем регистре')
print(sentences.str.lower())

print('Взять первое слово')
print(sentences.str.split().str[0])

print('Взять начало текста')
print(sentences.str[:10])

print('Замена по регулряному вырожению')
print(sentences.str.replace('(\d)+', '[NUMBER]', regex=True))

### pd.DataFrame

`pd.DataFrame` представляет собой таблицу, различные столбцы которой могут иметь различные типы данных.
Каждая колонка и каждлая строка представляет собой `pd.Series`

In [ ]:
df = pd.DataFrame({
    'age': [23, 30, 23],
    'car': ['Volvo', 'BMW', 'Mercedes'],
    'married': [True, False, True],
})

#### Базовые методы
Одинаковые для `pd.Series` и `pd.DataFrame`, однако, для `pd.DataFrame` можно указывать вдоль какой оси сделать (если требуется).

- `min`
- `max`
- `sum`
- `cumsum`
- `head`
- `describe`
- `sample`
- `isnull`
- `replace` — позволяет заменить по словарю, другому `pd.Series`
- `map`

### Input/Output
Поддерживает популярные форматы:
- CSV
- JSON
- Excel
- [Apache Parquet](https://parquet.apache.org/)


Функции `pd.read_{FORMAT}` и `pd.DataFrame.to_{FORMAT}`

### Индексация
- `.loc` - обращение по ключу. Можно передать несколько индексов, тогда будет выбраны строка и столбец
- `.iloc` - обращение по номеру элемента (зависит от того, в каком порядке лежат элементы, может быть нестабильным)
- `[]` - выбрать колонку


Так же можно выбирать по булевой маске или срезы


#### Multiindex
Элементом индекса может быть кортеж. В этом случае можно делать срезы только по первой компоненте, например.


#### Работа с индексом
- `.set_index` - устанвовить индексом аргмуент (например, можно передать имя колонки)
- `.reset_index` - сбросить индекс и занумеровать числами от 0 до числа строк
- `.sort_index` - расположить строки в порядке индекса

### Группировка и агрегаты
У рядов и таблиц есть методы, которые позволяют вычислять агргеаты: сумму, среднее, минимум, максимум, ...

Но часто нужно посчитать статистику внутри какой-либо группу, для этого можно сгруппировать ряд или таблицу

### [Adult](https://archive.ics.uci.edu/dataset/2/adult)

Датасет, содержащий 32561 запись о жителях США. Собран в 1994 году.

**Цель**: предсказать будет ли заработок превышать 50000$ или нет

In [ ]:
columns = [
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education-num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'native-country',
    'target',
]


adult = pd.read_csv('../../data/adult/adult.data', header=None)
adult.columns = columns

adult.head()

In [ ]:
adult.loc[3:5]

In [ ]:
adult.loc[:, 'age':'education']

In [ ]:
adult['target'].unique()

In [ ]:
adult['binary_target'] = adult['target'].replace({' <=50K': 0, ' >50K': 1})

In [ ]:
adult.describe()

In [ ]:
adult['relationship'].value_counts()

In [ ]:
grouped = adult.groupby(['sex', 'workclass']).agg({
    'binary_target': ['count', 'mean'],
    'age': ['median'],
})

grouped.head()

In [ ]:
grouped.loc[' Female': (' Male', ' Federal-gov')]

## Визуализация

### Почему визуализация важна?
Квартет Энскомба

In [ ]:
import pandas as pd


anscombe = pd.read_csv('../../data/anscombe_quartet.csv')
anscombe.head()

In [ ]:
anscombe.agg(['mean', 'std'])

In [ ]:
anscombe_corr = anscombe.corr()
for i in range(1, 5):
    corr = anscombe_corr.loc[(f'x{i}', f'y{i}')]
    print(f'corr(x{i},y{i})={corr}')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


n_groups = 4
row_size = 2
fig, axes = plt.subplots(
    n_groups // row_size,
    row_size,
    figsize=(6, 6),
    sharex=True,
    sharey=True,
)
for i in range(1, 5):
    row = (i - 1) // row_size
    column = (i - 1) % row_size
    ax = axes[row][column]
    x = anscombe[f'x{i}']
    y = anscombe[f'y{i}']
    ax.scatter(x, y)
    ax.set_title(f'Группа {i}')

fig.suptitle('Квартет Энскомба')
plt.subplots_adjust()

### Библиотеки для визуализации
* [Matplotlib](https://matplotlib.org/) — одна из первых библиотек, до сих пор широко используется. Канонически импортируется как 
    ```python
    import matloblib.pyplot as plt
    ```
    
* [Seaborn](https://seaborn.pydata.org/) — надстройка на matplotlib, упрощает многие рутинные операции.
Канонически импортируется как 
```python
import seaborn as sns
```

* [Bokeh](https://bokeh.org/) - библиотека, позволяющая стороить интерактивные графики и виджеты

* [Plotly](https://plotly.com/graphing-libraries/) — библиотека с упором на интерактивность

### Типы графиков

#### Гистограмма

In [ ]:
import seaborn as sns
sns.set()
sns.displot(adult['age'], kde=True)

In [ ]:
sns.displot(
    data=adult,
    x='age',
    col='sex',
    hue='target',
)

#### Точечные графики (диаграммы рассеивания)

In [ ]:
sns.scatterplot(
    data=adult,
    x='age',
    y='capital-gain',
)

In [ ]:
sns.pairplot(adult)

#### Ящичные диограмым (box-plot)

In [ ]:
sns.boxplot(
    data=adult,
    x='education-num',
    y='age',
)

In [ ]:
sns.violinplot(
    data=adult,
    x='education-num',
    y='age',
)

In [ ]:
adult.columns

In [ ]:
adult.pivot_table(values=['binary_target'], index=['sex', 'education'], aggfunc='mean')

### Ссылки
1. [Сборник](https://github.com/HorusHeresyHeretic/Pandas_Practice) тетрадок по Pandas
1. [Еще тетрадки по pandas](https://github.com/jupyter/jupyter/wiki#a-gallery-of-interesting-jupyter-notebooks)
1. [Pandas для больших данных](https://habr.com/ru/companies/ruvds/articles/442516/) 
